In [0]:
"""
In this notebook , I try LDA(Latent Dirichlet Allocation) 
for content downloaded from wikipedia.
I am downloading prefectures of Japan and states of USA.
Then give these vocublary to LDA and ask to make 2 topics.
"""
import gensim
from gensim import corpora

In [8]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [0]:
import wikipedia
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from gensim.corpora import Dictionary

import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation)
lemma = WordNetLemmatizer()
def clean(doc):
  stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
  punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
  normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
  return normalized

def download_from_wiki(keywords):
  splits = keywords.split(",")
  sentences = []
  for split in splits:
    try:
      sentences.append(wikipedia.summary( split ))
    except:
      print("error ",split) 
  return sentences  

In [65]:
japan_states = "Hokkaidō,Aomori,Iwate,Miyagi,Akita,Yamagata,Fukushima,Ibaraki,Tochigi,Gunma,Saitama,Chiba,Tōkyō,Kanagawa,Niigata,Toyama,Ishikawa,Fukui,Yamanashi,Nagano,Gifu,Shizuoka,Aichi,Mie,Shiga,Kyōto,Ōsaka,Hyōgo,Nara,Wakayama,Tottori,Shimane,Okayama,Hiroshima,Yamaguchi,Tokushima,Kagawa,Ehime,Kōchi,Fukuoka,Saga,Nagasaki,Kumamoto,Ōita,Miyazaki,Kagoshima,Okinawa"

japan_sentences = download_from_wiki( japan_states )

usa_states = "Alabama,Alaska,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Florida,Georgia,Hawaii,Idaho,Illinois,Indiana,Iowa,Kansas,Kentucky,Louisiana,Maine,Maryland,Massachusetts,Michigan,Minnesota,Mississippi,Missouri,Montana,Nebraska,Nevada,New Hampshire,New Jersey,New Mexico,New York,North Carolina,North Dakota,Ohio,Oklahoma,Oregon,Pennsylvania,Rhode Island,South Carolina,South Dakota,Tennessee,Texas,Utah,Vermont,Virginia,Washington,West Virginia,Wisconsin,Wyoming"
usa_sentences = download_from_wiki( usa_states )

/usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.6/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


error  Miyagi
error  Fukushima
error  Tochigi
error  Saitama
error  Niigata
error  Toyama
error  Ishikawa
error  Fukui
error  Yamanashi
error  Nagano
error  Shizuoka
error  Mie
error  Nara
error  Wakayama
error  Yamaguchi
error  Georgia
error  Indiana
error  New York
error  Rhode Island
error  Washington


In [0]:
doc_clean_japan = [clean(doc).split() for doc in japan_sentences]
doc_clean_usa = [clean(doc).split() for doc in usa_sentences]


In [0]:
 def run_lda(vocublary,num_topic,num_pass):
  dictionary = Dictionary(vocublary)
  doc_term_matrix = [dictionary.doc2bow(doc) for doc in vocublary]
  Lda = gensim.models.ldamodel.LdaModel
  model = Lda(doc_term_matrix, num_topics=num_topic, id2word = dictionary, passes=num_pass)
  #print(model.print_topics(num_topics=num_topic, num_words=6))
  return model

In [58]:
model1 = run_lda(doc_clean_usa,2)
model1.print_topics(-1)

[(0,
  '0.043*"state" + 0.010*"area" + 0.008*"u" + 0.008*"united" + 0.007*"largest" + 0.007*"city" + 0.006*"south" + 0.006*"american" + 0.006*"new" + 0.005*"populous"'),
 (1,
  '0.041*"state" + 0.011*"new" + 0.010*"united" + 0.008*"north" + 0.007*"area" + 0.006*"mexico" + 0.006*"city" + 0.006*"virginia" + 0.006*"largest" + 0.006*"west"')]

In [61]:
model1 = run_lda(doc_clean_usa+doc_clean_japan,2,500)
model1.print_topics(-1)

[(0,
  '0.027*"prefecture" + 0.018*"city" + 0.017*"japan" + 0.008*"area" + 0.007*"capital" + 0.007*"population" + 0.006*"largest" + 0.006*"world" + 0.006*"region" + 0.005*"japanese"'),
 (1,
  '0.046*"state" + 0.009*"area" + 0.009*"united" + 0.008*"new" + 0.007*"largest" + 0.007*"north" + 0.007*"city" + 0.007*"u" + 0.006*"american" + 0.006*"south"')]

In [63]:
model1 = run_lda(doc_clean_usa+doc_clean_japan,2,3000)
model1.print_topics(-1)

[(0,
  '0.005*"china" + 0.005*"missouri" + 0.005*"world" + 0.005*"film" + 0.004*"kansa" + 0.004*"japanese" + 0.003*"massachusetts" + 0.003*"akita" + 0.003*"japan" + 0.003*"osaka"'),
 (1,
  '0.041*"state" + 0.013*"city" + 0.011*"area" + 0.010*"prefecture" + 0.008*"largest" + 0.008*"united" + 0.007*"new" + 0.007*"population" + 0.007*"north" + 0.007*"region"')]

In [64]:
#as the number of pass increases keywords get less importance and more refined
model1 = run_lda(doc_clean_usa+doc_clean_japan,2,5000)
model1.print_topics(-1)

[(0,
  '0.045*"state" + 0.009*"area" + 0.009*"united" + 0.008*"new" + 0.007*"largest" + 0.007*"north" + 0.007*"city" + 0.007*"u" + 0.006*"american" + 0.006*"south"'),
 (1,
  '0.028*"prefecture" + 0.019*"city" + 0.018*"japan" + 0.008*"area" + 0.007*"capital" + 0.007*"population" + 0.006*"japanese" + 0.006*"largest" + 0.006*"region" + 0.005*"located"')]